In [2]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig

/Users/maoyiyun/opt/anaconda3/envs/DL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import numpy as np
from scipy.special import softmax

In [4]:
import pandas as pd
import re

In [12]:
UK_df = pd.read_csv('data/ElectricCarUK-comments.csv')
UK_df.head()

,author,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_name,subreddit_id
0,jchannon,"This looks great, will check it out. Surprise...","<div class=""md""><p>This looks great, will chec...",1548355257,None,None,eev2r1k,None,t3_ajbhjw,t3_ajbhjw,/r/ElectricCarUK/comments/ajbhjw/ive_made_an_a...,None,2,None,ElectricCarUK,t5_o9449
1,[deleted],Super thrilled you’ve done this! 1 small thing...,"<div class=""md""><p>Super thrilled you’ve done ...",1548368952,None,None,eevop91,None,t3_ajbhjw,t3_ajbhjw,/r/ElectricCarUK/comments/ajbhjw/ive_made_an_a...,None,2,None,ElectricCarUK,t5_o9449
2,antpk,Love it but unfortunately I'm on Android.,"<div class=""md""><p>Love it but unfortunately I...",1548331143,None,None,eeu6fez,None,t3_ajbhjw,t3_ajbhjw,/r/ElectricCarUK/comments/ajbhjw/ive_made_an_a...,None,2,None,ElectricCarUK,t5_o9449
3,FillingUpTheDatabase,From 10th June 2021 this will be the pricing f...,"<div class=""md""><p>From 10th June 2021 this wi...",1622556382,None,1622557075,h070epp,TRUE,t3_npusqp,t3_npusqp,/r/ElectricCarUK/comments/npusqp/bp_pulse_pric...,None,2,None,ElectricCarUK,t5_o9449
4,happysleepygoat,I’m confused by this email. \n\nMy workplace h...,"<div class=""md""><p>I’m confused by this email....",1622559742,None,None,h077ld1,None,t3_npusqp,t3_npusqp,/r/ElectricCarUK/comments/npusqp/bp_pulse_pric...,None,1,None,ElectricCarUK,t5_o9449


In [72]:
evcharging_df = pd.read_csv('data/evcharging-comments.csv')
evcharging_df.head()

,author,body,body_html,created_utc,distinguished,edited,id,is_submitter,link_id,parent_id,permalink,saved,score,stickied,subreddit_name,subreddit_id
0,JacksonDWalter,Something similar to this occurred at a loft m...,"<div class=""md""><p>Something similar to this o...",1698195708,None,None,k6bxtk8,None,t3_17fryug,t3_17fryug,/r/evcharging/comments/17fryug/mind_your_idle_...,None,41,None,evcharging,t5_vprim
1,nxtiak,What a fool. I've unplugged people that finish...,"<div class=""md""><p>What a fool. I&#39;ve unplu...",1698193918,None,1698194101,k6btdk7,None,t3_17fryug,t3_17fryug,/r/evcharging/comments/17fryug/mind_your_idle_...,None,94,None,evcharging,t5_vprim
2,Illcatchyoubeerbaron,My office has some free chargers for employees...,"<div class=""md""><p>My office has some free cha...",1698209135,None,None,k6crcgj,None,t3_17fryug,t3_17fryug,/r/evcharging/comments/17fryug/mind_your_idle_...,None,20,None,evcharging,t5_vprim
3,odinsen251a,New high score!,"<div class=""md""><p>New high score!</p>\n</div>",1698205954,None,None,k6cll3q,None,t3_17fryug,t3_17fryug,/r/evcharging/comments/17fryug/mind_your_idle_...,None,5,None,evcharging,t5_vprim
4,simplystriking,"I ain't unplugging shit, they wanna waste othe...","<div class=""md""><p>I ain&#39;t unplugging shit...",1698225184,None,None,k6dd0lv,None,t3_17fryug,t3_17fryug,/r/evcharging/comments/17fryug/mind_your_idle_...,None,5,None,evcharging,t5_vprim


In [80]:
RealTesla_df = pd.read_csv('data/RealTesla-comments.csv')



In [86]:
Bolt_df = pd.read_csv('data/BoltEV-comments.csv')


In [5]:
def drop_useless_columns(df):
    # Drop unnecessary columns
    df = df[['author','body','score']]
    # Rename some columns
    df.rename(columns={'body': 'text'}, inplace=True)
    return df

In [6]:
def drop_bot_comment(df): 
    # Get the list of bots comments
    bot_list = [
        'Decronym',
        'stabbot',
        'stabbot_crop',
        'DeepFryBot',
        'gifreversingbot',
        'vredditshare',
        'VredditDownloader',
        'morejpeg_auto',
        'gifendore',
        'r2tg_bot',
        'WololoBot',
        'tippr',
        'RemindMeBot',
        'profanitycounter',
        'Eminem_Bot'
    ]

    # Check number of posts is bot author
    print(f"Number of bot comments: {df[df['author'].isin(bot_list)].shape[0]}")
    df = df[~df['author'].isin(bot_list)]
    df = df.drop(columns=['author'])
    return df

In [7]:
def remove_multiple_spaces(text):
    # Count
    num_substitutions = len(re.findall(r'\s+', text)) - 1 # Minus 1 because we are replacing with 1 space
    # Remove
    cleaned_text = re.sub(r'\s+', ' ', text)
    return cleaned_text, num_substitutions
def remove_df_extra_spaces(df):
    df['text'], counts = zip(*df['text'].apply(remove_multiple_spaces))
    print(f"Number of removed extra spaces: {sum(counts)}")
    return df

In [8]:
def replace_abbr(text):    
    abbr_mapper = {
        # Reddit abbreviations & Slangs
        'Alt': 'Alternative Reddit account',
        'AMA': 'Ask me anything',
        'AMAA': 'Ask me almost anything',
        'Benned': 'Banned',
        'Brony': 'Male fan of My Little Pony',
        'Cakeday': 'Birthday',
        'Circlejerk': 'Elitist group',
        'DAE': 'Does anyone else',
        'Ent': 'Pot smoker',
        'ETA': 'Edited to add',
        'F7U12': 'FU',
        'Fap': 'Masturbate',
        '[FIXED]': 'Remix of an original post',
        'FTA': 'From the article',
        'FTFY': 'Fixed That For You',
        'GW': 'Gone wild',
        'Hivemind': 'Collective',
        'IAMA': 'I Am A',
        'IMO': 'In My Opinion',
        'IMHO': 'In my honest opinion',
        'IIRC': 'If i recall correctly',
        'ITT': 'In this thread',
        'Karma': 'Reddit score',
        'Karmawhore': 'Desperate for reddit points',
        'Meta-sub': 'Subreddits talking about Reddit',
        'Meta-subreddits': 'Subreddits talking about Reddit',
        'MIC': 'More in comments',
        'Mod': 'Moderator',
        'MRA': 'Mens rights activist',
        'Neckbeard': 'Dirty reddit user',
        'Ninjaedit': 'sneaky edit',
        'Novelty account': 'joke account',
        'NSFW': 'Not safe for work',
        'NSFL': 'Not safe for life',
        'OP': 'Original Poster',
        'Orangered': 'Unread messages',
        'Power user': 'User with high reddit score',
        'Pun thread': 'Chain of punny comments',
        'Reddiquette': 'Rules of reddit',
        'RES': 'Reddit enhancement suite',
        'RTFA': 'Read the fucking article',
        'Shadow-ban': 'Silent ban',
        'Shitpost': 'Trash post',
        'Sockpuppet': 'Alternate reddit account',
        'SJW': 'Social Justice Warrior',
        'SRD': 'Subreddit drama',
        'SRS': 'Shit reddit says',
        'Sub': 'Subreddit',
        'TIL': 'Today I learned',
        'TL;DR': 'Too Long Didnt read',
        'TLDR': 'Too Long Didnt read',
        'WIP': 'Work in progress',
        'X-post': 'Crosspost',
        'Xpost': 'Crosspost',
        'wh[o]+sh': 'Dont get the joke',
        
        # Replace model names with their full brand names as well, Electric car domain specific abbreviations & slangs
        
        
        # Other common internet abbr & slangs
        'LOL': 'Laugh out loud',
        'TTYL': 'Talk to you later',
        'ASAP': 'As soon as possible',
        'FYI': 'For your information',
        'JK': 'Just kidding',
        'IDC': 'I dont care',
        'FTW': 'For the win',
        'LMAO': 'Laughing my ass off',
        'LMFAO': 'Laughing my fucking ass off',
        'BFF': 'Best friend forever',
        'MFW': 'My face when',
        'TFW': 'That feeling when',
        'G2G': 'Got to go',
        'MSG': 'Message',
    }
    
    # Standardize all cases in mapper
    abbr_mapper =  {key.lower(): val for key, val in abbr_mapper.items()}

    # Regex to match every key
    # r'\b' -> Matches from start to finish
    # (?:) -> Capturing group
    # '|'.join(mapper) -> Loop and form pattern of all keys
    pattern = r'\b(?:' + r'|'.join(re.escape(abbr) for abbr in [k.lower() for k in abbr_mapper.keys()]) + r')\b'
    # Compile and replace
    pattern = re.compile(pattern, re.IGNORECASE)
    # Count
    num_substitutions = len(re.findall(pattern, text))
    replaced_text = pattern.sub(lambda match: abbr_mapper[match.group(0).lower()], text)

    return replaced_text, num_substitutions

def df_replace_abbr(df):
    # Apply to text field of df
    df['text'], counts = zip(*df['text'].apply(replace_abbr))
    print(f"Number of replaced abbreviations: {sum(counts)}")
    return df

In [9]:
# Function to count words in a sentence
def count_words(sentence):
    words = sentence.split()
    return len(words)

def select_comments(df, min, max):
    # Add a column with the number of words in each sentence
    df['Word Count'] = df['text'].apply(count_words)
    # Select rows with sentences containing 4 to 20 words
    selected_rows = df[(df['Word Count'] >= min) & (df['Word Count'] <= max)]
    selected_rows = selected_rows.drop(columns=['Word Count'])
    print("Number of selected rows:", len(selected_rows))
    return selected_rows

In [10]:
def preprocess(df, min, max):
    print("Number of rows in original dataset: ", len(df))
    df = drop_useless_columns(df)
    df = drop_bot_comment(df)
    df = remove_df_extra_spaces(df)
    df = df_replace_abbr(df)
    df = select_comments(df, min, max)
    return df

In [54]:
processed_UK_df = preprocess(UK_df, 5, 450)
processed_UK_df.head(10)

Number of rows in original dataset:  73
Number of bot comments: 0
Number of removed extra spaces: 4561
Number of replaced abbreviations: 0
Number of selected rows: 67


,text,score
0,"This looks great, will check it out. Surprised...",2
1,Super thrilled you’ve done this! 1 small thing...,2
2,Love it but unfortunately I'm on Android.,2
3,From 10th June 2021 this will be the pricing f...,2
4,I’m confused by this email. My workplace has “...,1
5,"I have to admit, I have 0 idea how much I spen...",1
6,This is what I’ve been waiting for. The sooner...,2
7,This one looks like it has 43kw AC but the one...,2
9,I’ve answered your questionnaire. Good luck wi...,2
12,Seems poorly planned not having chargers a vis...,2


In [11]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
def prediction(text):
#text = "Covid cases are increasing fast!"
#text = preprocess(text)
#encoded_input = tokenizer(processed_UK_df['text'].tolist(), max_length=514, padding=True, truncation=True, return_tensors="pt")
    #print(text)
    encoded_input = tokenizer(text, max_length=500, padding=True, truncation=True, return_tensors="pt")
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = config.id2label[ranking[0]]
    return scores, label


In [13]:
text = "Covid cases are increasing fast!"
scores, label = prediction(text)
print(scores)
print(label)

[0.7235768  0.2286789  0.04774429]
negative


In [60]:
scores_list, labels = zip(*processed_UK_df['text'].apply(prediction))


In [58]:
#encoded_input = tokenizer(processed_UK_df['text'].tolist(), max_length=500, padding=True, truncation=True, return_tensors="pt")
#print(text)
#encoded_input = tokenizer(text, max_length=500, padding=True, truncation=True, return_tensors="pt")
#output = model(**encoded_input)

In [79]:
def classification(df,name):
    print("start classification")
    scores_list, labels = zip(*df['text'].apply(prediction))
    s_list = np.array(scores_list)
    df["neg_score"] = s_list[:, 0]
    df["neu_score"] = s_list[:, 1]
    df["pos_score"] = s_list[:, 2]
    df["label"] = labels
    df.to_csv(f"classification/{name}.csv",index=False, encoding='utf-8')
    return df

In [64]:
processed_UK_df.head(10)

,text,score,neg_score,neu_score,pos_score,label
0,"This looks great, will check it out. Surprised...",2,0.008371,0.025596,0.966033,positive
1,Super thrilled you’ve done this! 1 small thing...,2,0.055226,0.147884,0.796890,positive
2,Love it but unfortunately I'm on Android.,2,0.356486,0.270438,0.373076,positive
3,From 10th June 2021 this will be the pricing f...,2,0.045968,0.898498,0.055533,neutral
4,I’m confused by this email. My workplace has “...,1,0.201317,0.765437,0.033245,neutral
5,"I have to admit, I have 0 idea how much I spen...",1,0.732036,0.231553,0.036411,negative
6,This is what I’ve been waiting for. The sooner...,2,0.005787,0.053832,0.940381,positive
7,This one looks like it has 43kw AC but the one...,2,0.498156,0.331384,0.170460,negative
9,I’ve answered your questionnaire. Good luck wi...,2,0.005441,0.090637,0.903922,positive
12,Seems poorly planned not having chargers a vis...,2,0.776949,0.208614,0.014437,negative


In [70]:
processed_UK_df.to_csv("classification/UK_comment.csv",index=False, encoding='utf-8')

In [78]:
processed_evcharging_df = preprocess(evcharging_df,5,500)
predict_evcharging = classification(processed_evcharging_df, "evcharging")
predict_evcharging.head(15)

Number of rows in original dataset:  1339
Number of bot comments: 0
Number of removed extra spaces: 48243


/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/823002335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'body': 'text'}, inplace=True)


Number of replaced abbreviations: 63
Number of selected rows: 1225


,text,score,neg_score,neu_score,pos_score,label
0,Something similar to this occurred at a loft m...,41,0.687681,0.299403,0.012917,negative
1,What a fool. I've unplugged people that finish...,94,0.666745,0.299159,0.034096,negative
2,My office has some free chargers for employees...,20,0.374554,0.534225,0.091221,neutral
4,"I ain't unplugging shit, they wanna waste othe...",5,0.835935,0.153498,0.010567,negative
5,I 100% support this fee and wish it were even ...,3,0.141329,0.302278,0.556393,positive
6,Laugh out loud! They got charged what they des...,2,0.623661,0.293311,0.083028,negative
7,My apartment charges $10 an hour to idle… this...,2,0.915550,0.076569,0.007881,negative
8,"If I was in a particularly snarktastic mood, I...",2,0.706766,0.277994,0.015240,negative
9,Airport by me has charging stations with six h...,2,0.449910,0.483379,0.066711,neutral
10,"Pfft, looks like my trucks weekly gas bill. La...",2,0.460984,0.348875,0.190141,negative


In [84]:
processed_RealTesla_df = preprocess(RealTesla_df,5,500)
predict_RealTesla = classification(processed_RealTesla_df, "RealTesla")


Number of rows in original dataset:  10786
Number of bot comments: 0
Number of removed extra spaces: 216375


/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/823002335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'body': 'text'}, inplace=True)


Number of replaced abbreviations: 673
Number of selected rows: 9320
start classification


In [85]:
# Get value counts for the 'label' column
value_counts = predict_RealTesla['label'].value_counts()

# Calculate the total number of values in the column
total_count = len(predict_RealTesla)

# Calculate the percentage and count for each unique value
percentage_counts = (value_counts / total_count) * 100
absolute_counts = value_counts

# Display the results
print("Percentage of each unique value:")
print(percentage_counts)
print("\nAbsolute count of each unique value:")
print(absolute_counts)

Percentage of each unique value:
negative    58.744635
neutral     31.030043
positive    10.225322
Name: label, dtype: float64

Absolute count of each unique value:
negative    5475
neutral     2892
positive     953
Name: label, dtype: int64


In [122]:
popular_comment_Tesla = predict_RealTesla[((predict_RealTesla['score'] >= 17)&(predict_RealTesla['label'] == "negative"))\
                                           |((predict_RealTesla['score'] >= 9)&(predict_RealTesla['label'] == "neutral"))\
                                            |((predict_RealTesla['score'] >= 3)&(predict_RealTesla['label'] == "positive"))]
print(popular_comment_Tesla.shape)
# Get value counts for the 'label' column
value_counts = popular_comment_Tesla['label'].value_counts()

# Calculate the total number of values in the column
total_count = len(popular_comment_Tesla)

# Calculate the percentage and count for each unique value
percentage_counts = (value_counts / total_count) * 100
absolute_counts = value_counts

# Display the results
print("Percentage of each unique value:")
print(percentage_counts)
print("\nAbsolute count of each unique value:")
print(absolute_counts)

(1289, 6)
Percentage of each unique value:
negative    43.444531
neutral     32.660978
positive    23.894492
Name: label, dtype: float64

Absolute count of each unique value:
negative    560
neutral     421
positive    308
Name: label, dtype: int64


In [128]:
popular_comment_Tesla[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
popular_comment_Tesla.to_csv("classification/popular_comment_Telsa.csv",index=False, encoding='utf-8')


/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/3449500806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_comment_Tesla[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/3449500806.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_comment_Tesla[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykern

In [87]:
processed_Bolt_df = preprocess(Bolt_df,5,500)
predict_Bolt = classification(processed_Bolt_df, "BoltEV")


Number of rows in original dataset:  4439
Number of bot comments: 0
Number of removed extra spaces: 111377


/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/823002335.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'body': 'text'}, inplace=True)


Number of replaced abbreviations: 237
Number of selected rows: 3879
start classification


In [88]:
# Get value counts for the 'label' column
value_counts = predict_Bolt['label'].value_counts()

# Calculate the total number of values in the column
total_count = len(predict_Bolt)

# Calculate the percentage and count for each unique value
percentage_counts = (value_counts / total_count) * 100
absolute_counts = value_counts

# Display the results
print("Percentage of each unique value:")
print(percentage_counts)
print("\nAbsolute count of each unique value:")
print(absolute_counts)

Percentage of each unique value:
neutral     39.572055
negative    37.535447
positive    22.892498
Name: label, dtype: float64

Absolute count of each unique value:
neutral     1535
negative    1456
positive     888
Name: label, dtype: int64


In [126]:
popular_comment_Bolt = predict_Bolt[((predict_Bolt['score'] >= 4)&(predict_Bolt['label'] == "negative"))\
                                           |((predict_Bolt['score'] >= 4)&(predict_Bolt['label'] == "neutral"))\
                                            |((predict_Bolt['score'] >= 3)&(predict_Bolt['label'] == "positive"))]
print(popular_comment_Bolt.shape)
# Get value counts for the 'label' column
value_counts = popular_comment_Bolt['label'].value_counts()

# Calculate the total number of values in the column
total_count = len(popular_comment_Bolt)

# Calculate the percentage and count for each unique value
percentage_counts = (value_counts / total_count) * 100
absolute_counts = value_counts

# Display the results
print("Percentage of each unique value:")
print(percentage_counts)
print("\nAbsolute count of each unique value:")
print(absolute_counts)

(1196, 6)
Percentage of each unique value:
neutral     35.284281
negative    33.612040
positive    31.103679
Name: label, dtype: float64

Absolute count of each unique value:
neutral     422
negative    402
positive    372
Name: label, dtype: int64


In [129]:
popular_comment_Bolt[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
popular_comment_Bolt.to_csv("classification/popular_comment_Bolt.csv",index=False, encoding='utf-8')

/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/1452255117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_comment_Bolt[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_36576/1452255117.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  popular_comment_Bolt[["annotator 1","irony 1","remove 1","annotator 2","irony 2","remove 2"]] = ""
/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel

### Datasets to be labelled
2 datasets(subtopics): comments for BoltEV and comments for RealTesla. Existing columns: text, score(number of up-votes minus number of down-votes), softmax scores for all three lables, predicted label.

### Annotation instruction
2 annotators are needed, each of them need to do the following:
1. classify the comment as positive(1), neutral(0) or negative(-1) under the column annotater 1(2)
2. classify the comment as ironic(1) or unironic(0) under the column irony 1(2)
3. if you feel the comment lacks context to the extent that it is difficult to label, put 1 under the column remove 1(2)
4. if you feel the comment is not relevant to the subtopic, put 2 under the column remove 1(2)

Caveat: 
* Even you put 1 or 2 under column remove, still fill in the values for anotator and irony columns to the best of your knowledge. 
* Predicion results from the pre-trained model are available in the csv file. Try not to refer to them too much when doing the labelling. We use the annotators inputs as ground truths.
* We hope to have 1000 entries for each dataset. Currently each dataset has around 1.2k entries. That means we can accept around 200 entries to be removed for each dataset.  

### Model used for classification
A RoBERTa-base model trained on ~124M tweets from January 2018 to December 2021, and finetuned for sentiment analysis with the TweetEval benchmark.https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest The checkpoint is found on HuggingFace. No guarantee it will work well on our own dataset but it is the most relevant I can find. No training/finetuning is performed because we do not have labelled data at this time and very likely will not have the time to label extra data besides the 2 test datasets in the future.

### Innovation
Considering ensemble classification or sarcasm detection. For both the most feasible approach will still be using checkpoints on hugging face. The irony columns are labelled so we can experiment on sarcasm detection, if the results are too bad, may just switch to ensemble classification or other methods that do not require extra labelling. 

In [21]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [14]:
annotateddf = pd.read_csv('popular_comment_Bolt_annotate_Merged.csv')

In [18]:
Bolt_predicted = pd.read_csv('classification/popular_comment_Bolt.csv')

In [19]:
annotateddf['annotator_combined'] = np.where(annotateddf['annotator 1'] == annotateddf['annotator 2'], annotateddf['annotator 1'], np.nan)
valid_count = annotateddf['annotator_combined'].count()
print("valid count: ", valid_count)
annotateddf.dropna(subset=['annotator_combined'], inplace=True)


valid count:  1068


In [20]:
annotateddf['annotator_combined'] = annotateddf['annotator_combined'].astype(str)
sentiment_mapping = {'-1.0': 'negative', '0.0': 'neutral', '1.0': 'positive'}

annotateddf['annotator_combined'] = annotateddf['annotator_combined'].replace(sentiment_mapping)

In [30]:
remove_count_1 = annotateddf['remove 1'].count()
print("remove 1:", remove_count_1)
remove_count_2 = annotateddf['remove 2'].count()
print("remove 2:", remove_count_1)

remove 1: 150
remove 2: 150


In [31]:
count_non_na = annotateddf[['remove 1', 'remove 2']].notna().all(axis=1).sum()
print("common remove: ", count_non_na)

common remove:  34


In [38]:
annotateddf = annotateddf[annotateddf['remove 1'].isna() | annotateddf['remove 2'].isna()]
print(len(annotateddf))

1034


In [39]:
annotation_df = annotateddf[["text","annotator_combined"]]

In [40]:
merged_df = pd.merge(Bolt_predicted, annotation_df, on='text', how='inner')
merged_df.head()

,text,score,neg_score,neu_score,pos_score,label,annotator 1,irony 1,remove 1,annotator 2,irony 2,remove 2,annotator_combined
0,An Ultium based Bolt with NACS and faster char...,219,0.004940,0.088701,0.906359,positive,NaN,NaN,NaN,NaN,NaN,NaN,neutral
1,They had to have something smaller than the Eq...,141,0.378837,0.540830,0.080333,neutral,NaN,NaN,NaN,NaN,NaN,NaN,negative
2,Yes! I just came back to post this very same t...,55,0.003039,0.012690,0.984271,positive,NaN,NaN,NaN,NaN,NaN,NaN,positive
3,Yes!!! So excited! Was thinking I was going to...,52,0.017278,0.055458,0.927264,positive,NaN,NaN,NaN,NaN,NaN,NaN,positive
4,Interesting that they dropped the EV/EUV namep...,21,0.032266,0.881413,0.086321,neutral,NaN,NaN,NaN,NaN,NaN,NaN,neutral


In [41]:
accuracy = accuracy_score(merged_df['annotator_combined'], merged_df['label'])
precision = precision_score(merged_df['annotator_combined'], merged_df['label'], average='weighted')
recall = recall_score(merged_df['annotator_combined'], merged_df['label'], average='weighted')
f1 = f1_score(merged_df['annotator_combined'], merged_df['label'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8191489361702128
Precision: 0.8217577552345009
Recall: 0.8191489361702128
F1 Score: 0.8198113471835082


In [42]:
accuracy = accuracy_score(merged_df['annotator_combined'], merged_df['label'])
precision = precision_score(merged_df['annotator_combined'], merged_df['label'], average='micro')
recall = recall_score(merged_df['annotator_combined'], merged_df['label'], average='micro')
f1 = f1_score(merged_df['annotator_combined'], merged_df['label'], average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8191489361702128
Precision: 0.8191489361702128
Recall: 0.8191489361702128
F1 Score: 0.8191489361702128


In [44]:
from transformers import pipeline
classifier = pipeline('zero-shot-classification', model='roberta-large-mnli')

2024-04-09 22:59:13.767067: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
config.json: 100%|██████████| 688/688 [00:00<00:00, 45.3kB/s]
model.safetensors: 100%|██████████| 1.43G/1.43G [03:11<00:00, 7.44MB/s]
Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.wei

In [77]:
text_list = merged_df['text'].tolist()


In [80]:
len(text_list)

1034

In [84]:
def mnli_predictin(text):
    candidate_labels = ['positive', 'neutral', 'negative']
    result = classifier(text, candidate_labels)
    scores_list = result["scores"]
    labels_list = result["labels"]
    chosen_labels = labels_list[0]
    return scores_list, labels_list, chosen_labels

In [ ]:
candidate_labels = ['positive', 'neutral', 'negative']
result = classifier(text_list, candidate_labels)

In [66]:
result

[{'sequence': 'today is a bad day',
  'labels': ['negative', 'neutral', 'positive'],
  'scores': [0.9906196594238281, 0.007247406058013439, 0.0021329601295292377]}]

In [75]:
scores_list = [d['scores'] for d in result if 'scores' in d]
labels_list = [d['labels'] for d in result if 'labels' in d]
chosen_labels = list(zip(*labels_list))[0]

In [76]:
labels_list

['negative', 'positive']

In [85]:
scores_list, labels_list, labels = zip(*merged_df['text'].apply(mnli_predictin))

In [86]:
merged_df["mnli_label"] = labels

In [118]:
accuracy = accuracy_score(merged_df['annotator_combined'], merged_df['mnli_label'])
precision = precision_score(merged_df['annotator_combined'], merged_df['mnli_label'], average='weighted')
recall = recall_score(merged_df['annotator_combined'], merged_df['mnli_label'], average='weighted')
f1 = f1_score(merged_df['annotator_combined'], merged_df['mnli_label'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.6537717601547389
Precision: 0.6937598452290512
Recall: 0.6537717601547389
F1 Score: 0.6152323371340676


In [88]:
vaders = pd.read_csv('VadersTextBlobCombinedData.csv')


/var/folders/zd/n573jzn975s_mpdy3pm9tt1h0000gn/T/ipykernel_21964/3355802776.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  vaders = pd.read_csv('VadersTextBlobCombinedData.csv')


In [90]:
vaders = vaders[["text","vader_sentiment"]]
combined_df = pd.merge(vaders, merged_df, on='text', how='inner')


In [91]:
vote_counts = combined_df[["label","mnli_label","vader_sentiment"]].apply(pd.Series.value_counts, axis=1)

In [92]:
consensus_predictions = vote_counts.idxmax(axis=1)
combined_df['consensus_prediction'] = consensus_predictions

### majority voting for reberta, mnli,and vader

In [93]:
accuracy = accuracy_score(combined_df['annotator_combined'], combined_df['consensus_prediction'])
precision = precision_score(combined_df['annotator_combined'], combined_df['consensus_prediction'], average='weighted')
recall = recall_score(combined_df['annotator_combined'], combined_df['consensus_prediction'], average='weighted')
f1 = f1_score(combined_df['annotator_combined'], combined_df['consensus_prediction'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7350096711798839
Precision: 0.7565219647149439
Recall: 0.7350096711798839
F1 Score: 0.7197461219568052


In [98]:

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    top_k = None
)

In [108]:
distilled_student_sentiment_classifier ("I love this movie and i would watch it again and again!")
tokenizer_kwargs = {'padding':True,'truncation':True,'max_length':512}


In [109]:
def distilled_prediction(text):
    #print(text)
    result = distilled_student_sentiment_classifier(text,**tokenizer_kwargs)
    label = [item['label'] for item in result[0]][0]
    scores = [item['score'] for item in result[0]]
    return scores, label

In [110]:
distilled_scores, distilled_labels = zip(*combined_df['text'].apply(distilled_prediction))

In [111]:
combined_df["distilled_label"] = distilled_labels

In [112]:
accuracy = accuracy_score(combined_df['annotator_combined'], combined_df['distilled_label'])
precision = precision_score(combined_df['annotator_combined'], combined_df['distilled_label'], average='weighted')
recall = recall_score(combined_df['annotator_combined'], combined_df['distilled_label'], average='weighted')
f1 = f1_score(combined_df['annotator_combined'], combined_df['distilled_label'], average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5135396518375241
Precision: 0.5053416348267816
Recall: 0.5135396518375241
F1 Score: 0.4538706506308806


In [113]:
vote_counts = combined_df[["mnli_label","vader_sentiment","distilled_label"]].apply(pd.Series.value_counts, axis=1)
new_consensus_predictions = vote_counts.idxmax(axis=1)
combined_df['new_consensus_predictions'] = new_consensus_predictions

In [119]:
accuracy = accuracy_score(combined_df['annotator_combined'], combined_df['new_consensus_predictions'])
precision = precision_score(combined_df['annotator_combined'], combined_df['new_consensus_predictions'], average='micro')
recall = recall_score(combined_df['annotator_combined'], combined_df['new_consensus_predictions'], average='micro')
f1 = f1_score(combined_df['annotator_combined'], combined_df['new_consensus_predictions'], average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.6005802707930368
Precision: 0.6005802707930368
Recall: 0.6005802707930368
F1 Score: 0.6005802707930368


In [120]:
vote_counts = combined_df[["label","mnli_label","vader_sentiment","distilled_label"]].apply(pd.Series.value_counts, axis=1)
consensus_predictions4 = vote_counts.idxmax(axis=1)
combined_df['consensus_predictions4'] = consensus_predictions4

In [121]:
accuracy = accuracy_score(combined_df['annotator_combined'], combined_df['consensus_predictions4'])
precision = precision_score(combined_df['annotator_combined'], combined_df['consensus_predictions4'], average='micro')
recall = recall_score(combined_df['annotator_combined'], combined_df['consensus_predictions4'], average='micro')
f1 = f1_score(combined_df['annotator_combined'], combined_df['consensus_predictions4'], average='micro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.6876208897485493
Precision: 0.6876208897485493
Recall: 0.6876208897485493
F1 Score: 0.6876208897485493


In [123]:
combined_df.to_csv("majorityVoting.csv",index = False)